# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847793312951                   -0.70    4.5   88.7ms
  2   -7.852408853165       -2.34       -1.53    1.0   18.7ms
  3   -7.852610056311       -3.70       -2.55    1.0   18.5ms
  4   -7.852645458011       -4.45       -2.78    2.2   25.0ms
  5   -7.852646067791       -6.21       -2.87    1.0   18.7ms
  6   -7.852646666629       -6.22       -3.87    1.0   18.5ms
  7   -7.852646685414       -7.73       -4.67    1.5   21.3ms
  8   -7.852646686702       -8.89       -5.15    1.5   21.9ms
  9   -7.852646686727      -10.61       -5.65    1.2   20.0ms
 10   -7.852646686728      -11.88       -5.58    1.5   21.5ms
 11   -7.852646686730      -11.75       -6.30    1.0   19.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.847795881277                   -0.70           4.8    426ms
  2   -7.852562090579       -2.32       -1.62   0.80    2.0    2.29s
  3   -7.852641730069       -4.10       -2.70   0.80    1.0    227ms
  4   -7.852646514667       -5.32       -3.40   0.80    1.8   21.6ms
  5   -7.852646681352       -6.78       -4.41   0.80    1.5   20.4ms
  6   -7.852646686608       -8.28       -4.85   0.80    2.5   25.6ms
  7   -7.852646686726       -9.93       -5.48   0.80    1.0   17.9ms
  8   -7.852646686730      -11.39       -6.54   0.80    1.5   20.2ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.201144214242                   -1.06    4.27s
  2   -1.353070864366        0.41       -0.68    107ms
  3   -3.771485095358        0.38       -0.43   33.6ms
  4   -4.885333205778        0.05       -0.58   33.6ms
  5   -6.706978066753        0.26       -0.68   33.8ms
  6   -7.347936593986       -0.19       -1.15   33.5ms
  7   -7.614660081520       -0.57       -1.38   25.5ms
  8   -7.726994418788       -0.95       -1.73   25.9ms
  9   -7.793756630164       -1.18       -1.72   25.7ms
 10   -7.823808012029       -1.52       -2.14   25.6ms
 11   -7.841368719819       -1.76       -2.48   25.5ms
 12   -7.847576816153       -2.21       -2.47   25.4ms
 13   -7.850225227551       -2.58       -2.80   25.3ms
 14   -7.851638747579       -2.85       -2.90   25.3ms
 15   -7.852191407453       -3.26       -3.19   25.4ms
 16   -7.852474159784       -3.55       -3.50   25.8ms
 17   -7.85

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847773094015                   -0.70    4.5   35.5ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645850405                   -1.63    13.7s
  2   -7.852646686730       -6.08       -3.69    3.62s
  3   -7.852646686730      -13.25       -7.20    130ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.204344341301999e-7
|ρ_newton - ρ_scfv| = 3.7038909581097414e-7
|ρ_newton - ρ_dm|   = 1.0122318708635521e-9
